# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
## Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS project1b 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
## Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('project1b')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

#### For table music_library_by_sessions, the column session_id and item_in_session were used as a partition key because the queries will filter by this columns. 

In [8]:
## Create query for create table

query_create_table_1 = "CREATE TABLE IF NOT EXISTS music_library_by_sessions "
query_create_table_1 = query_create_table_1 + "(session_id int, item_in_session int, artist text, song_title text, song_length float, PRIMARY KEY (session_id, item_in_session))"

## Excute query create table

try:
    session.execute(query_create_table_1)
except Exception as e:
    print(e)

In [9]:
# Create query 1
query1 = """ SELECT artist, song_title, song_length 
             FROM music_library_by_sessions 
             WHERE session_id = %s AND item_in_session = %s """

#### Let's insert the data into the table

In [10]:
## We have provided part of the code to set up the CSV file.
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Assign the INSERT statements into the `query` variable
        query_insert = "INSERT INTO music_library_by_sessions (session_id, item_in_session, artist, song_title, song_length)"
        query_insert = query_insert + " VALUES (%s, %s, %s, %s, %s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query_insert, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [11]:
## Add in the SELECT statement to verify the data was entered into the table  
## Excute query 1 with value sessionId = 338, and itemInSession = 4

try:
    rows = session.execute(query1, (338,4))
except Exception as e:
    print(e)
    
## Print data
for row in rows:
    print("Artist: "+row.artist, "\nSong Title: "+row.song_title, "\nSong Length: "+str(row.song_length))

Artist: Faithless 
Song Title: Music Matters (Mark Knight Dub) 
Song Length: 495.30731201171875


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

#### For table users_details, the column user_id and session_id were used as a partition key because the queries will filter by this column.  The column item_in_session was used as clustering columns to help make up a unique key.

In [12]:
## Create query for create table
## Data sort by item_in_session

query_create_table_2 = "CREATE TABLE IF NOT EXISTS users_details"
query_create_table_2 = query_create_table_2 + "(user_id int, session_id int, item_in_session int, artist text, song_title text, first_name text, last_name text, PRIMARY KEY ((user_id, session_id), item_in_session)) WITH CLUSTERING ORDER BY (item_in_session DESC);"

## Excute query create table

try:
    session.execute(query_create_table_2)
except Exception as e:
    print(e)                

In [13]:
## Crete query 2

query2 = """ SELECT artist, song_title, first_name, last_name
             FROM users_details
             WHERE user_id  = %s AND session_id = %s """

#### Let's insert the data into the table

In [14]:
## We have provided part of the code to set up the CSV file.
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Assign the INSERT statements into the `query` variable
        query_insert_2 = "INSERT INTO users_details (user_id, session_id , item_in_session, artist, song_title, first_name, last_name)"
        query_insert_2 = query_insert_2 + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query_insert_2, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

#### Do a SELECT to verify that the data have been inserted into each table

In [15]:
## Add in the SELECT statement to verify the data was entered into the table

## Excute query 2 with value userid = 10, sessionid = 182
try:
    rows = session.execute(query2, (10,182))
except Exception as e:
    print(e)
    
## Print data
for row in rows:
    print("\nName of artist: "+row.artist, "\nSong (sorted by itemInSession): "+row.song_title, "\nFirst name: "+row.first_name, "\nLast name: "+row.last_name)


Name of artist: Lonnie Gordon 
Song (sorted by itemInSession): Catch You Baby (Steve Pitron & Max Sanna Radio Edit) 
First name: Sylvie 
Last name: Cruz

Name of artist: Sebastien Tellier 
Song (sorted by itemInSession): Kilometer 
First name: Sylvie 
Last name: Cruz

Name of artist: Three Drives 
Song (sorted by itemInSession): Greece 2000 
First name: Sylvie 
Last name: Cruz

Name of artist: Down To The Bone 
Song (sorted by itemInSession): Keep On Keepin' On 
First name: Sylvie 
Last name: Cruz


### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

#### For table users_listen_song, the column song_title was used as a partition key because the queries will filter by this column.  The column user_id  was used as clustering columns to help make up a unique key.

In [16]:
## Create query for create table

query_create_table_3 = "CREATE TABLE IF NOT EXISTS users_listen_song "
query_create_table_3 = query_create_table_3 + "(song_title text, user_id int, first_name text, last_name text, PRIMARY KEY ((song_title), user_id));"

## Excute query create table

try:
    session.execute(query_create_table_3)
except Exception as e:
    print(e)      

In [17]:
## Crete query 3

query3 = """ SELECT song_title, first_name, last_name
             FROM users_listen_song
             WHERE song_title = %s """

#### Let's insert the data into the table

In [ ]:
## We have provided part of the code to set up the CSV file.
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Assign the INSERT statements into the `query` variable
        query_insert_3 = "INSERT INTO users_listen_song (song_title, user_id, first_name, last_name)"
        query_insert_3 = query_insert_3 + " VALUES (%s, %s, %s, %s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query_insert_3, (line[9], int(line[10]), line[1], line[4]))

#### Do a SELECT to verify that the data have been inserted into each table

In [ ]:
## Add in the SELECT statement to verify the data was entered into the table

## Excute query 3 every user name (first and last) listen song 'All Hands Against His Own'

try:
    rows = session.execute(query3, ('All Hands Against His Own', ))
except Exception as e:
    print(e)
    
for row in rows:
    print("\nFirst name: "+row.first_name, "\nLast name: "+row.last_name,  "\nSong: "+row.song_title,)

### Drop the tables before closing out the sessions

In [ ]:
## Drop the table before closing out the sessions
session.execute("DROP TABLE IF EXISTS music_library_by_sessions")
session.execute("DROP TABLE IF EXISTS users_details")
session.execute("DROP TABLE IF EXISTS users_listen_song")

### Close the session and cluster connection¶

In [ ]:
session.shutdown()
cluster.shutdown()